<a href="https://colab.research.google.com/github/Sottix99/ADM-HW5/blob/main/HW5_ale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data

## Data Preprocessing

In [1]:
import pandas as pd
import networkx as nx
import string 
import matplotlib.pyplot as plt
import scipy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
edges = pd.read_csv('/content/drive/MyDrive/hw 5 ADM/edges.csv')

In [4]:
hero_network = pd.read_csv('/content/drive/MyDrive/hw 5 ADM/hero-network.csv')

In [5]:
nodes = pd.read_csv('/content/drive/MyDrive/hw 5 ADM/nodes.csv')

Some of the heroes' names in 'hero-network.csv' are not found in 'edges.csv'. This inconsistency exists for the following reasons:

Some heroes' names in 'hero-netowrk.csv' have extra spaces at the end of their names compared to their names in 'edges.csv'.
Some heroes' names in 'hero-netowrk.csv' have an extra '/' at the end of their names compared to their names in 'edges.csv'.
The hero name 'SPIDER-MAN/PETER PARKER' in 'edges.csv' has been changed to 'SPIDER-MAN/PETER PAR' in 'hero-network.csv' due to a string length limit in 'hero-network.csv'.

In [6]:
#Removing extra spaces at the end of the hero name
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip())
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip())

In [7]:
#Replace 'SPIDER-MAN/PETER PAR' with 'SPIDER-MAN/PETER PARKER'
hero_network.replace({'SPIDER-MAN/PETER PAR': 'SPIDER-MAN/PETER PARKER'}, inplace = True)

In [8]:
#Removing extra '/'
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip('/'))
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip('/'))
edges['hero'] = edges['hero'].apply(lambda x: x.strip('/'))

In [9]:
#After this we need to remove again the empty spaces at the end
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip())
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip())
edges['hero'] = edges['hero'].apply(lambda x: x.rstrip())

In [10]:
set(hero_network['hero1']).union(set(hero_network['hero2'])) == set(edges['hero'])

False

In [11]:
(set(hero_network['hero1']).union(set(hero_network['hero2']))).difference(set(edges['hero']))

set()

In [12]:
len(set(edges['hero']).difference(set(hero_network['hero1']).union(set(hero_network['hero2']))))

18

### Removing duplicates

In [13]:
hero_network.drop((hero_network[hero_network['hero1'] == hero_network['hero2']]).index, inplace = True)

### Save datasets

In [14]:
hero_network.to_csv('/content/drive/MyDrive/hw 5 ADM/hero_net_clean.csv', index=False)

In [15]:
edges.to_csv('/content/drive/MyDrive/hw 5 ADM/edges_clean.csv', index=False)

In [16]:
nodes.to_csv('/content/drive/MyDrive/hw 5 ADM/nodes_clean.csv', index=False)

## Graphs setup

#### First

In [17]:
hero_network

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...
574462,COLOSSUS II/PETER RA,CALLISTO
574463,CALLISTO,ROGUE
574464,CALLISTO,CALIBAN
574465,CALIBAN,ROGUE


In [18]:
hero_net_g=hero_network.groupby(hero_network.columns.tolist(),as_index=False).size()
hero_net_g

,hero1,hero2,size
0,24-HOUR MAN/EMMANUEL,"FROST, CARMILLA",1
1,24-HOUR MAN/EMMANUEL,KILLRAVEN/JONATHAN R,1
2,24-HOUR MAN/EMMANUEL,M'SHULLA,1
3,3-D MAN/CHARLES CHAN,ANGEL/WARREN KENNETH,1
4,3-D MAN/CHARLES CHAN,ANT-MAN II/SCOTT HAR,1
...,...,...,...
224094,ZZZAX,"RODRIGUEZ, DEBRA",1
224095,ZZZAX,"ROSS, GEN. THADDEUS",2
224096,ZZZAX,"SUMMERS, NATHAN CHRI",1
224097,ZZZAX,TIGRA/GREER NELSON,1


we have the new df 

In [19]:
hero_net_g['size'].describe()

count    224099.000000
mean          2.553492
std           7.285278
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        1267.000000
Name: size, dtype: float64

In [20]:
hero_net_g['size'].sort_values()[-100:]

186191     133
148855     133
218821     134
78102      135
91072      135
          ... 
196190     365
85416      366
195966     382
124735     627
142471    1267
Name: size, Length: 100, dtype: int64

In [21]:
G_1 = nx.from_pandas_edgelist(hero_net_g, 'hero1', 'hero2' , edge_attr='size') # dobbiamo scegliere la metrica giusta

In [22]:
#nx.draw(G_1) # mi da errore, forse sono troppi nodi?
#plt.show()

#### Second

In [23]:
nodes

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero
...,...,...
19085,"ZOTA, CARLO",hero
19086,ZOTA,hero
19087,ZURAS,hero
19088,ZURI,hero


In [24]:
edges

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251
...,...,...
96099,ZZZAX,H2 326
96100,ZZZAX,H2 327
96101,ZZZAX,M/CP 8/4
96102,ZZZAX,PM 47


First, we create the normal graph from edges.csv

In [25]:
G_2 = nx.from_pandas_edgelist(edges, 'hero', 'comic')

Then we create two different list that contains only the name of heroes or comics

In [26]:
n_h=nodes[nodes['type'] == 'hero']['node'].tolist()

In [27]:
n_c=nodes[nodes['type'] == 'comic']['node'].tolist()

Now we merge the two lists in a dictionary 

In [28]:
dict_heroes_comics = {key: 'hero' if key in n_h else 'comic' for key in n_c + n_h}

Finally we add at each node of the graph its type

In [29]:
nx.set_node_attributes(G_2, dict_heroes_comics, 'type')

In [30]:
#nx.draw(G_2) # mi da errore, forse sono troppi nodi?
#plt.show()

# 2. Backend Implementation

In [50]:
def get_topN(N):
  series=edges['hero'].value_counts()[0:N]
  series_list=series.index.tolist()
  return series_list

In [56]:
all_heroes=len(edges['hero'].value_counts())
all_heroes

In [53]:
get_topN(10)

['SPIDER-MAN/PETER PARKER',
 'CAPTAIN AMERICA',
 'IRON MAN/TONY STARK',
 'THING/BENJAMIN J. GR',
 'THOR/DR. DONALD BLAK',
 'HUMAN TORCH/JOHNNY S',
 'MR. FANTASTIC/REED R',
 'HULK/DR. ROBERT BRUC',
 'WOLVERINE/LOGAN',
 'INVISIBLE WOMAN/SUE']

In [33]:
edges

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251
...,...,...
96099,ZZZAX,H2 326
96100,ZZZAX,H2 327
96101,ZZZAX,M/CP 8/4
96102,ZZZAX,PM 47


In [47]:
edges['hero'].value_counts()[0:10].index.tolist()

['SPIDER-MAN/PETER PARKER',
 'CAPTAIN AMERICA',
 'IRON MAN/TONY STARK',
 'THING/BENJAMIN J. GR',
 'THOR/DR. DONALD BLAK',
 'HUMAN TORCH/JOHNNY S',
 'MR. FANTASTIC/REED R',
 'HULK/DR. ROBERT BRUC',
 'WOLVERINE/LOGAN',
 'INVISIBLE WOMAN/SUE']

In [32]:
nx.density(G_1)

0.008107160063285336

In [ ]:
def ex_features(G, G_type, N = all_heroes):
  
  # The network's density
  density = nx.density(G)
  # The average degree of the network
  avg_degree=sum(G.degree().values())/float(len(G))
